In [2]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Prueba de hipótesis

Para determinar qué prueba de hipótesis realizar con este conjunto de datos, primero debemos identificar la pregunta o la afirmación que deseamos investigar. Aquí planteamos algunos ejemplos de preguntas o afirmaciones que probaremos para entender en que consiste la prueba de hipótesis:

1. **Hipótesis sobre los ingresos promedio en función de la educación:**
   - Hipótesis nula (H0): No hay diferencia en los ingresos promedio entre las personas con educación básica 4 años y las personas con educación secundaria.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en los ingresos promedio entre estos dos grupos.
   - Prueba de hipótesis: Se podría realizar una prueba t de Student para comparar los ingresos promedio de estos dos grupos.

2. **Hipótesis sobre la duración de la llamada telefónica en función del estado civil:**
   - Hipótesis nula (H0): No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en la duración promedio de las llamadas telefónicas entre estos dos grupos.
   - Prueba de hipótesis: También se podría utilizar una prueba t de Student para comparar la duración promedio de las llamadas entre los dos grupos de estado civil.


La función `ttest_ind` se utiliza para realizar una prueba t de Student independiente entre dos muestras (grupos) para comparar si sus medias son estadísticamente diferentes. Aquí está la sintaxis básica:

```python
ttest_ind(a, b, axis, equal_var, nan_policy=)
```


Uno de los parámetros que tenemos en el `ttest_ind` es el `equal_var`, donde le tenemos que decir, si nuestras categorías a comparar tienen varianzas iguales o no. Por lo tanto, lo primero que tendremos que hacer es evaluarlo, para eso tenemos dos test: **Prueba de Levene**
y la **Prueba de Bartlett**. 

> En ambos casos, si el p-valor < 0.05 podemos concluir que las varianzas son diferentes entre los grupos. Si el p-valor > 0.05, no podemos afirmar que las varianzas son diferentes.

In [3]:
# lo primero que hacemos es cargar el dataframe que vamos a usar para esta lección
df = pd.read_csv("./bank-additional_clean.csv", index_col = 0)
df.head(2)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores
1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,57.0,services,married,high school,NaN,No,No,telephone,149,1,NaN,0,nonexistent,1.1,93.994,-36.4,NaN,5191,no,14-septiembre-2016,34.601,-83.923,septiembre,2016.0,Mediana edad


In [9]:
# vamos a definir una función para poder hacer el t-test de Student
def prueba_hipotesis(*args):
    """
    Realiza una prueba de hipótesis para comparar grupos.
    1. Primero verifica si los datos son normales usando el test de Shapiro-Wilk o Kolmogorov-Smirnov.
    2. Si los datos son normales, usa Bartlett para probar igualdad de varianzas. Si no son normales, usa Levene.
    3. Si las varianzas son iguales, usa el t-test de Student; si no, usa la versión de Welch.
    4. Si los datos no son normales, usa el test de Mann-Whitney (alternativa no paramétrica).

    Parámetros:
    *args: listas o arrays con los datos de cada grupo.

    Retorna:
    dict con resultados del test de normalidad, varianza e hipótesis.
    """
    
    # Verificar si hay al menos dos grupos
    if len(args) < 2:
        raise ValueError("Se necesitan al menos dos conjuntos de datos para realizar la prueba.")
    
    # Comprobar normalidad en cada grupo
    normalidad = []
    for grupo in args:
        if len(grupo) > 50:
            p_valor_norm = stats.kstest(grupo, 'norm').pvalue  # Kolmogorov-Smirnov si n > 50
        else:
            p_valor_norm = stats.shapiro(grupo).pvalue  # Shapiro-Wilk si n <= 50
        normalidad.append(p_valor_norm > 0.05)

    datos_normales = all(normalidad)  # True si todos los grupos son normales

    # Prueba de igualdad de varianzas
    if datos_normales:
        p_valor_varianza = stats.bartlett(*args).pvalue  # Test de Bartlett si los datos son normales
    else:
        p_valor_varianza = stats.levene(*args, center="median").pvalue  # Test de Levene si no son normales

    varianzas_iguales = p_valor_varianza > 0.05

    # Aplicar el test adecuado
    if datos_normales:
        if varianzas_iguales:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
            test_usado = "t-test de Student (varianzas iguales)"
        else:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
            test_usado = "t-test de Welch (varianzas desiguales)"
    else:
        t_stat, p_valor = stats.mannwhitneyu(*args)
        test_usado = "Mann-Whitney U (prueba no paramétrica)"

    # Nivel de significancia
    alfa = 0.05

    # Resultados
    resultado = {
        "Test de Normalidad": normalidad,
        "Datos Normales": datos_normales,
        "p-valor Varianza": p_valor_varianza,
        "Varianzas Iguales": varianzas_iguales,
        "Test Usado": test_usado,
        "Estadístico": t_stat,
        "p-valor": p_valor,
        "Conclusión": "Rechazamos H0 (Diferencias significativas)" if p_valor < alfa else "No se rechaza H0 (No hay diferencias significativas)"
    }

    # Imprimir resultados de manera más clara
    print("\n📊 **Resultados de la Prueba de Hipótesis** 📊")
    print(f"✅ Test de Normalidad: {'Sí' if datos_normales else 'No'}")
    print(f"   - Normalidad por grupo: {normalidad}")
    print(f"✅ Test de Varianza: {'Iguales' if varianzas_iguales else 'Desiguales'} (p = {p_valor_varianza:.4f})")
    print(f"✅ Test aplicado: {test_usado}")
    print(f"📉 Estadístico: {t_stat:.4f}, p-valor: {p_valor:.4f}")
    print(f"🔍 Conclusión: {resultado['Conclusión']}\n")

    return resultado

In [10]:

# filtramos los datos para obtener dos grupos: educación básica 4 años y educación secundaria
grupo_educacion_basica = df[df['education'] == 'basic 4y']["income"]
grupo_educacion_secundaria = df[df['education'] == 'high school']["income"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_educacion_basica, grupo_educacion_secundaria )



📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = 0.0104)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
📉 Estadístico: 21172740.0000, p-valor: 0.0504
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': 0.010397688688198759,
 'Varianzas Iguales': False,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'Estadístico': 21172740.0,
 'p-valor': 0.05035248597730047,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}

In [11]:
# vamos a probar la segunda hipótesis que tenemos planteada
# No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.

# lo primero que tenemos que hacer es crear dos DataFrames nuevos, uno para saber el tiempo de las llamadas a personas casadas y otro para personas solteras
grupo_casadas = df[df["marital"] == "married"]["duration"]
grupo_solteras = df[df["marital"] == "single"]["duration"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_casadas, grupo_solteras )


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Iguales (p = 0.0862)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
📉 Estadístico: 156542600.5000, p-valor: 0.4142
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': 0.08623883710573196,
 'Varianzas Iguales': True,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'Estadístico': 156542600.5,
 'p-valor': 0.4141512349135199,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}